# Environment

In [2]:
!pip install -r /content/drive/MyDrive/text_mining/multilingual-summarization-20250312/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install mecab-python3 rouge unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 44.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 289.2 kB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=87d7fe4fdf1f1098cd7055e65ebf742f6b4e24b01c5e1e0878157cc75c19da23
  Stored in directory: /root/.cache/pip/wheels/b7/fd/e9/ea4459b868e6d2902e8d80e82dbacb6203e05b3b3a58c64966
Successfully built unidic-lite


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kazetof (llm-summarization-fbt3r5) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
from huggingface_hub import login

# Replace with your actual Hugging Face token
hf_token = ""

# Log in to Hugging Face
login(token=hf_token)

# mbart

In [5]:
import os

import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_dataset, load_from_disk
from transformers import Trainer, TrainingArguments

# 使用するモデルとトークナイザーのロード
model_name = "facebook/mbart-large-50"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [6]:
# データセットの読み込み

dataset_path = "/content/drive/MyDrive/text_mining/hf_dataset_from_quiscale"
language = "ja"
dataset = load_from_disk(os.path.join(dataset_path, language))
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

In [7]:
def generate_summary(text: str, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(model.device)
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# summarize sample text
sample_text = "Your article text here."
print(generate_summary(sample_text, model, tokenizer))

Your article text here.


In [8]:
import MeCab

class MeCabTokenizer(object):
  """rouge-score用のMeCabを用いたTokenizerクラス

  Args:
      use_stemmer (bool, optional): Trueの場合、単語の原型で分割する. Defaults to False.
  """
  def __init__(self, use_stemmer=False):
    self._stemmer = use_stemmer

    self.tagger = MeCab.Tagger()
    self.wakati = MeCab.Tagger("-Owakati")


  def tokenize(self, text):
    if self._stemmer:
        node = self.tagger.parseToNode(text)
        original_forms = []
        while node:
            feature = node.feature.split(",")
            original_forms.append(feature[6])
            node = node.next

        return original_forms

    else:
      return self.wakati.parse(text).split()

In [9]:
from rouge import Rouge
import numpy as np

def calculate_rouge_scores(generated, reference):
    rouge = Rouge()
    scores = rouge.get_scores(generated, reference)[0]

    return {
        "rouge-1": scores["rouge-1"]["f"],
        "rouge-2": scores["rouge-2"]["f"],
        "rouge-l": scores["rouge-l"]["f"],
    }


In [10]:
def calculate_rouge_scores_for_japanese(generated, reference):
  mecab_tokenizer = MeCabTokenizer()

  generated_sep = mecab_tokenizer.tokenize(generated)
  reference_sep = mecab_tokenizer.tokenize(reference)

  reference_space = " ".join(reference_sep)
  generated_space = " ".join(generated_sep)

  rouge_score = calculate_rouge_scores(generated_space, reference_space)
  return rouge_score

In [11]:

def calculate_avg_rouge_scores(val_dataset, model, tokenizer):
  max_samples = 10
  indices = list(range(len(val_dataset)))
  sample_indices = np.random.choice(
      indices,
      min(max_samples, len(val_dataset)),
      replace=False
  )

  rouge_scores = []
  for idx in sample_indices:
      print("idx : ", idx)
      text = val_dataset[int(idx)]["text"]
      generated = generate_summary(text, model, tokenizer)
      reference = val_dataset[int(idx)]["summary"]

      score = calculate_rouge_scores_for_japanese(generated, reference)
      rouge_scores.append(score)
      print(score)

  # Calculate average scores
  avg_rouge_scores = {
      metric: np.mean([score[metric] for score in rouge_scores])
      for metric in ["rouge-1", "rouge-2", "rouge-l"]
  }
  return avg_rouge_scores

In [12]:
avg_rouge_scores = calculate_avg_rouge_scores(val_dataset, model, tokenizer)
print("avg : ", avg_rouge_scores)

idx :  305
{'rouge-1': 0.24137930555885861, 'rouge-2': 0.07446808043232261, 'rouge-l': 0.24137930555885861}
idx :  359
{'rouge-1': 0.04255318999119768, 'rouge-2': 0.00987654175448886, 'rouge-l': 0.035460991409637405}
idx :  135
{'rouge-1': 0.1987577594382934, 'rouge-2': 0.08270676246452621, 'rouge-l': 0.173913038941399}
idx :  152
{'rouge-1': 0.18461537961538477, 'rouge-2': 0.041666661680230636, 'rouge-l': 0.15384614884615402}
idx :  5
{'rouge-1': 0.3055555513854596, 'rouge-2': 0.17174514847645442, 'rouge-l': 0.23148147731138555}
idx :  298
{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}
idx :  473
{'rouge-1': 0.21935483464391264, 'rouge-2': 0.07575757208448136, 'rouge-l': 0.20645160883746105}
idx :  490
{'rouge-1': 0.1891891842996714, 'rouge-2': 0.04166666202222274, 'rouge-l': 0.1486486437591309}
idx :  450
{'rouge-1': 0.2935779766551637, 'rouge-2': 0.09756097065734708, 'rouge-l': 0.18348623353589782}
idx :  396
{'rouge-1': 0.2608695607490549, 'rouge-2': 0.10416666234399131, 'rouge-l

In [13]:
def tokenize_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]

    # tokenize input and label
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=150)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# apply tokenizer for dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# setting for training
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=200,
    evaluation_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    fp16=True,
    load_best_model_at_end=True
)

# setting Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

# finetuning
trainer.train()

# save model
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")




Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-5601e94a7aba>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kazetof (llm-summarization-fbt3r5) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,2.395100,1.914365
400,1.984300,1.825243
600,1.751400,1.805267
800,1.568400,1.772010
1000,1.588400,1.746961
1200,1.312300,1.787930
1400,1.310000,1.769377


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/sentencepiece.bpe.model',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [14]:
avg_rouge_scores_finetuned = calculate_avg_rouge_scores(val_dataset, model, tokenizer)
print("avg : ", avg_rouge_scores_finetuned)

idx :  362
{'rouge-1': 0.46280991287480366, 'rouge-2': 0.2413793058627296, 'rouge-l': 0.31404958229629126}
idx :  73
{'rouge-1': 0.48809523323412707, 'rouge-2': 0.32411066708119174, 'rouge-l': 0.45238094751984126}
idx :  375
{'rouge-1': 0.38509316390571363, 'rouge-2': 0.22857142516352041, 'rouge-l': 0.28571428191813597}
idx :  155
{'rouge-1': 0.32388663568588244, 'rouge-2': 0.2193211448689405, 'rouge-l': 0.299595137710174}
idx :  104
{'rouge-1': 0.37179486726824457, 'rouge-2': 0.18644067351228105, 'rouge-l': 0.2948717903451677}
idx :  395
{'rouge-1': 0.436170207861589, 'rouge-2': 0.14577258991032663, 'rouge-l': 0.2127659525424401}
idx :  378
{'rouge-1': 0.6947368371501386, 'rouge-2': 0.41958041463543455, 'rouge-l': 0.5263157845185596}
idx :  124
{'rouge-1': 0.3308823488194745, 'rouge-2': 0.16629213085155925, 'rouge-l': 0.2426470547018275}
idx :  68
{'rouge-1': 0.3741007149485016, 'rouge-2': 0.20370369935570998, 'rouge-l': 0.3165467581139693}
idx :  451
{'rouge-1': 0.44247787148249673, 

debug japanese

In [ ]:
text = val_dataset[0]["text"]
# ref = val_dataset[0]["summary"]
ref = "株式会社ナムコは、1955年に有限会社中村製作所として設立され、1977年に社名をナムコに変更した日本のゲームメーカーおよびアミューズメント施設運営企業であった。1980年代には『パックマン』など多くのヒットゲームを生み出し、その後も『鉄拳』や『リッジレーザー』など多くの人気ゲームシリーズを発表した。2005年にバンダイと経営統合し、2006年にバンダイナムコゲームスに社名変更した。アミューズメント部門は2006年に独立し、2018年にバンダイナムコアミューズメントに社名変更した。ナムコは日本のゲーム産業史上重要な役割を果たした。そのブランドは現在でも後継企業で使用されている。"
summary = generate_summary(text, model, tokenizer)

In [ ]:
print(summary)

株式会社ナムコは、バンダイナムコエンターテインメントとバンダイナムコアミューズメントの本社を運営するゲームセンター「namco」を展開する企業。1988年に設立され、1991年9月に上場。2005年に廃止。本社は東京都大田区矢口2丁目1番21号で、本社ビルは東京都大田区矢口の本社ビル。2021年にマンションが建設された。


In [ ]:
print(ref)

株式会社ナムコは、1955年に有限会社中村製作所として設立され、1977年に社名をナムコに変更した日本のゲームメーカーおよびアミューズメント施設運営企業であった。1980年代には『パックマン』など多くのヒットゲームを生み出し、その後も『鉄拳』や『リッジレーザー』など多くの人気ゲームシリーズを発表した。2005年にバンダイと経営統合し、2006年にバンダイナムコゲームスに社名変更した。アミューズメント部門は2006年に独立し、2018年にバンダイナムコアミューズメントに社名変更した。ナムコは日本のゲーム産業史上重要な役割を果たした。そのブランドは現在でも後継企業で使用されている。


In [ ]:
calculate_rouge_scores("株式会社ナムコ", "株式会社ナムコは")

{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}

In [ ]:
calculate_rouge_scores(summary, ref)

{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}